In [4]:
import json
import pandas as pd

In [5]:
CDR_01_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_01.json'))
CDR_02_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_02.json'))
CDR_03_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_03.json'))
CDR_04_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_04.json'))
CDR_05_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_05.json'))
CDR_06_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_06.json'))
CDR_07_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_07.json'))
CDR_08_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_08.json'))
CDR_09_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_09.json'))
CDR_10_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_10.json'))
CDR_11_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_11.json'))
CDR_12_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_12.json'))
CDR_13_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_13.json'))
CDR_14_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_14.json'))
CDR_15_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_15.json'))
CDR_16_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_16.json'))

CDR_01_json

{'meta': {'count': 2221, 'db_response_time_ms': 42, 'page': 1, 'per_page': 25},
 'results': [{'id': 'https://openalex.org/W2965283917',
   'display_name': 'Sorbents for the Direct Capture of CO <sub>2</sub> from Ambient Air',
   'authorships': [{'author_position': 'first',
     'author': {'id': 'https://openalex.org/A5076098034',
      'display_name': 'Xiaoyang Shi',
      'orcid': 'https://orcid.org/0000-0003-1097-9745'},
     'institutions': [{'id': 'https://openalex.org/I2802152550',
       'display_name': 'Earth Island Institute',
       'ror': 'https://ror.org/005y2a635',
       'country_code': 'US',
       'type': 'nonprofit'},
      {'id': 'https://openalex.org/I78577930',
       'display_name': 'Columbia University',
       'ror': 'https://ror.org/00hj8s172',
       'country_code': 'US',
       'type': 'education'},
      {'id': 'https://openalex.org/I55732556',
       'display_name': 'Arizona State University',
       'ror': 'https://ror.org/03efmqc40',
       'country_code': 

## Combine into a single CDR works list

In [6]:
works_CDR_01 = [(i['id'], i['open_access']['is_oa']) for i in CDR_01_json['results']]
works_CDR_02 = [(i['id'], i['open_access']['is_oa']) for i in CDR_02_json['results']]
works_CDR_03 = [(i['id'], i['open_access']['is_oa']) for i in CDR_03_json['results']]
works_CDR_04 = [(i['id'], i['open_access']['is_oa']) for i in CDR_04_json['results']]
works_CDR_05 = [(i['id'], i['open_access']['is_oa']) for i in CDR_05_json['results']]
works_CDR_06 = [(i['id'], i['open_access']['is_oa']) for i in CDR_06_json['results']]
works_CDR_07 = [(i['id'], i['open_access']['is_oa']) for i in CDR_07_json['results']]
works_CDR_08 = [(i['id'], i['open_access']['is_oa']) for i in CDR_08_json['results']]
works_CDR_09 = [(i['id'], i['open_access']['is_oa']) for i in CDR_09_json['results']]
works_CDR_10 = [(i['id'], i['open_access']['is_oa']) for i in CDR_10_json['results']]
works_CDR_11 = [(i['id'], i['open_access']['is_oa']) for i in CDR_11_json['results']]
works_CDR_12 = [(i['id'], i['open_access']['is_oa']) for i in CDR_12_json['results']]
works_CDR_13 = [(i['id'], i['open_access']['is_oa']) for i in CDR_13_json['results']]
works_CDR_14 = [(i['id'], i['open_access']['is_oa']) for i in CDR_14_json['results']]
works_CDR_15 = [(i['id'], i['open_access']['is_oa']) for i in CDR_15_json['results']]
works_CDR_16 = [(i['id'], i['open_access']['is_oa']) for i in CDR_16_json['results']]

works_CDR = works_CDR_01+works_CDR_02+works_CDR_03+works_CDR_04+works_CDR_05+works_CDR_06+works_CDR_07+works_CDR_08+works_CDR_09+works_CDR_10+works_CDR_11+works_CDR_12+works_CDR_13+works_CDR_14+works_CDR_15+works_CDR_16
print(len(works_CDR)) #400
df_works_CDR = pd.DataFrame(works_CDR)
df_works_CDR.head()

400


,0,1
0,https://openalex.org/W2965283917,False
1,https://openalex.org/W2965654741,False
2,https://openalex.org/W2963817167,False
3,https://openalex.org/W2960882361,False
4,https://openalex.org/W2972174901,False


### Separate into OA (open access) and NA (non-open access) objects

In [ ]:
df_CDR_OA = df_works_CDR.loc[df_works_CDR[1] == True]
df_CDR_NA = df_works_CDR.loc[df_works_CDR[1] == False]
print(len(df_CDR_OA), len(df_CDR_NA))

## Obtaining the OHW list

In [ ]:
OHW_OH_json = json.load(open('/workspaces/OHW_in_CDR/data/OHW/OHW_OH.json'))
OHW_OSH_json = json.load(open('/workspaces/OHW_in_CDR/data/OHW/OHW_OSH.json'))

works_OHW_OH = [(i['id'], i['open_access']['is_oa']) for i in OHW_OH_json['results']]
works_OHW_OSH = [(i['id'], i['open_access']['is_oa']) for i in OHW_OSH_json['results']]

works_OHW = works_OHW_OH+works_OHW_OSH
len(works_OHW) #50
df_works_OHW = pd.DataFrame(works_OHW)
df_works_OHW.head()

### Separate into OA (open access) and NA (non-open access) objects

In [ ]:
df_OHW_OA = df_works_OHW.loc[df_works_OHW[1] == True]
df_OHW_NA = df_works_OHW.loc[df_works_OHW[1] == False]
print(len(df_OHW_OA), len(df_OHW_NA))

## Open hardware cited in the title or abstract of co-authors’ works (2nd connection to OHW)

This is to answer the question of whether open hardware works added value to the work being published by being a work published by one or more co-authors, indicating likely awareness among the authors of open hardware because one of the co-authors has published OHW-related research.

In [ ]:
def get_records(results):
    records = []
    for data in results:
        work, work_id, oa, refs = data
        for obj in work:
            #print(obj)
            data = [obj['author']['id'], obj['author']['display_name'], obj['author']['orcid']]
            records.append([data, work_id, oa, refs])
    return records

def get_records_ohw(results):
    records = []
    for data in results:
        work, work_id, oa = data
        for obj in work:
            #print(obj)
            data = [obj['author']['id'], obj['author']['display_name'], obj['author']['orcid']]
            records.append([data, work_id, oa])
    return records

# get OA authors and NA authors
def get_OA_NA(lst):
    oa = []
    na = []
    for work in lst:
        if work[2] == True:
            oa.append((work[0], work[3]))
        else:
            na.append((work[0], work[3]))
    return oa, na

def get_OA_NA_ohw(lst):
    oa = []
    na = []
    for work in lst:
        if work[2] == True:
            oa.append((work[0]))
        else:
            na.append((work[0]))
    return oa, na

r01 = get_records([(i['authorships'], i['id'], i['open_access']['is_oa'], i['referenced_works']) for i in CDR_01_json['results']])
r02 = get_records([(i['authorships'], i['id'], i['open_access']['is_oa'], i['referenced_works']) for i in CDR_02_json['results']])
r03 = get_records([(i['authorships'], i['id'], i['open_access']['is_oa'], i['referenced_works']) for i in CDR_03_json['results']])
r04 = get_records([(i['authorships'], i['id'], i['open_access']['is_oa'], i['referenced_works']) for i in CDR_04_json['results']])
r05 = get_records([(i['authorships'], i['id'], i['open_access']['is_oa'], i['referenced_works']) for i in CDR_05_json['results']])
r06 = get_records([(i['authorships'], i['id'], i['open_access']['is_oa'], i['referenced_works']) for i in CDR_06_json['results']])
r07 = get_records([(i['authorships'], i['id'], i['open_access']['is_oa'], i['referenced_works']) for i in CDR_07_json['results']])
r08 = get_records([(i['authorships'], i['id'], i['open_access']['is_oa'], i['referenced_works']) for i in CDR_08_json['results']])
r09 = get_records([(i['authorships'], i['id'], i['open_access']['is_oa'], i['referenced_works']) for i in CDR_09_json['results']])
r10 = get_records([(i['authorships'], i['id'], i['open_access']['is_oa'], i['referenced_works']) for i in CDR_10_json['results']])
r11 = get_records([(i['authorships'], i['id'], i['open_access']['is_oa'], i['referenced_works']) for i in CDR_11_json['results']])
r12 = get_records([(i['authorships'], i['id'], i['open_access']['is_oa'], i['referenced_works']) for i in CDR_12_json['results']])
r13 = get_records([(i['authorships'], i['id'], i['open_access']['is_oa'], i['referenced_works']) for i in CDR_13_json['results']])
r14 = get_records([(i['authorships'], i['id'], i['open_access']['is_oa'], i['referenced_works']) for i in CDR_14_json['results']])
r15 = get_records([(i['authorships'], i['id'], i['open_access']['is_oa'], i['referenced_works']) for i in CDR_15_json['results']])
r16 = get_records([(i['authorships'], i['id'], i['open_access']['is_oa'], i['referenced_works']) for i in CDR_16_json['results']])
cdr_combined = r01+r02+r03+r04+r05+r06+r07+r08+r09+r10+r11+r12+r13+r14+r15+r16
print(len(cdr_combined)) #2508
cdr_oa_authors, cdr_na_authors = get_OA_NA(cdr_combined)
print(len(cdr_oa_authors), len(cdr_na_authors))

r_OH = get_records_ohw([(i['authorships'], i['id'], i['open_access']['is_oa']) for i in OHW_OH_json['results']])
r_OSH = get_records_ohw([(i['authorships'], i['id'], i['open_access']['is_oa']) for i in OHW_OSH_json['results']])
ohw_combined = r_OH+r_OSH
print(len(ohw_combined))
ohw_oa_authors, ohw_na_authors = get_OA_NA_ohw(ohw_combined)
print(len(ohw_oa_authors), len(ohw_na_authors))


### Look for matches between the CDR author IDs and the OHW author IDs

In [ ]:
cdr_oa_author_id_list = []
cdr_na_author_id_list = []
ohw_author_id_list = []

cdr_oa_ref_id_list = []
cdr_na_ref_id_list = []
ohw_work_id_list = []

for tup in cdr_oa_authors:
    cdr_oa_author_id_list.append(tup[0])
    cdr_oa_ref_id_list.append(tup[1])
for tup in cdr_na_authors:
    cdr_na_author_id_list.append(tup[0])
    cdr_na_ref_id_list.append(tup[1])
for tup in ohw_oa_authors:
    ohw_author_id_list.append(tup)
for tup in ohw_na_authors:
    ohw_author_id_list.append(tup)
for tup in works_OHW:
    ohw_work_id_list.append(tup[0])

cdr_ohw_oa_match_count = sum(el in cdr_oa_author_id_list for el in ohw_author_id_list)
cdr_ohw_na_match_count = sum(el in cdr_na_author_id_list for el in ohw_author_id_list)
print(cdr_ohw_oa_match_count, cdr_ohw_na_match_count)

## Open hardware cited in the title or abstract of the referenced works (3rd connection to OHW)

This is to answer the question of whether one or more open hardware works added value to the work being published by being included as a referenced work.

In [ ]:
print(len(cdr_oa_ref_id_list))
print(len(cdr_na_ref_id_list))
print(len(ohw_work_id_list))

cdr_ohw_oa_match_count = sum(el in cdr_oa_ref_id_list for el in ohw_work_id_list)
cdr_ohw_na_match_count = sum(el in cdr_na_ref_id_list for el in ohw_work_id_list)
print(cdr_ohw_oa_match_count, cdr_ohw_na_match_count)